In [4]:
import json
from pprint import pprint

In [91]:
#/Users/aditya/Documents/Spring_2019/BioAsq/7B/Batch1/BioASQ-task7bPhaseB-testset1.json
#trainining7b.json
with open('/Users/aditya/Documents/Spring_2019/BioAsq/7B/Batch5/BioASQ-task7bPhaseB-testset5.json') as f:
    data = json.load(f)



In [92]:

paragraphs = []
for quesion in data['questions']:
    if(quesion['type'] == "list"):
        answers = []
        answerContent = {}
        #text = quesion["exact_answer"][0]
        answer_start = 0
        #answerContent["text"] = text
        answerContent["answer_start"] = answer_start
        answers.append(answerContent)
        
        qas = []
        qasContent = {}
        id = quesion["id"]
        question = quesion["body"] 
        qasContent['question'] = question
        qasContent['id'] = id
        qasContent['answers'] = answers
        qas.append(qasContent)
        
        # context uing urls
        context = getContext(id) 
        #context = getConetextUsingSnippet(id) 
        # context uing snippets
        #context = getSnippets(id)
        paragraphContent = {}
        paragraphContent["context"] = context
        paragraphContent["qas"] = qas
        
        paragraphs.append(paragraphContent)


In [13]:
# ### testing
# paragraphs = []
# for quesion in data['questions']:
#     if(quesion['type'] == "factoid" or quesion['type'] == "list"):        
#         id = quesion["id"]
#         snippets = getSnippets(id)
#         for snippet in snippets: 
#             answers = []
#             answerContent = {}
#             #text = quesion["exact_answer"][0]
#             answer_start = 0
#             #answerContent["text"] = text
#             answerContent["answer_start"] = answer_start
#             answers.append(answerContent)

#             qas = []
#             qasContent = {}
#             id = quesion["id"]
#             question = quesion["body"] 
#             qasContent['question'] = question
#             qasContent['id'] = id
#             qasContent['answers'] = answers
#             qas.append(qasContent)

#             paragraphContent = {}
#             paragraphContent["context"] = snippet["text"]
#             paragraphContent["qas"] = qas
#             paragraphs.append(paragraphContent)


In [93]:
data = []
dataContent = {}
dataContent["title"] = "BioASQ6b" 
dataContent["paragraphs"] = paragraphs
data.append(dataContent)

finalDict = {}
finalDict["version"] = "BioASQ6b"
finalDict["data"] = data

In [94]:

#save the file in proper format
with open('/Users/aditya/Documents/Spring_2019/BioAsq/7B/Batch5/list_urls.json', 'w') as fp:
    #json.dump(finalDict, fp, sort_keys=True, indent=4)
    json.dump(finalDict, fp, sort_keys=False, indent=4)


### Generate Context 

In [68]:
import requests
import os
import xml.etree.ElementTree as ET

In [86]:
def generateUrlList(id):
    for quesion in data['questions']:
        if(quesion['type'] == "list" and quesion["id"] == id):
            urList = []
            for url in quesion["documents"]:
                urList.append(url)
    return(urList)            

In [87]:
#url = "http://www.ncbi.nlm.nih.gov/pubmed/18672707?report=xml&format=text"
def generateContentInXml(id):
    urList = generateUrlList(id)
    contentList = []
    for url in urList:
        url = url + "?report=xml&format=text"
        response = requests.get(url)
        s = response.text
        s = s.replace("&lt;","<")
        s = s.replace("&gt;",">")
        s = s.replace('<i>', '')
        s = s.replace('</i>', '')
        contentList.append(s)
    return(contentList)   
        

In [88]:
def generateContent(contentList):
    finaltext = ""
    for content in contentList:
        root = ET.fromstring(content)
        context = ""
        for node in root.iter():
            if(node.tag == "AbstractText"):    
                if(node.text != None):
                    context += node.text
        finaltext += context
    return(finaltext)    

In [89]:
def getContext(id):
    contextXml = generateContentInXml(id)
    #print(contextXml)
    context = generateContent(contextXml)
    return(context)

In [90]:
#56c1f01def6e394741000045
#52bf1b0a03868f1b06000009
#52bf19c503868f1b06000001
print(getContext("5c890ad575a4a5d21900000d"))

TypeError: list indices must be integers or slices, not str

### Context using snippet

In [53]:
def getConetextUsingSnippet(id):
    for quesion in data['questions']:
        if(quesion['type'] == "list" and quesion["id"] == id):
            context = ""
            for snippet in quesion["snippets"]:
                #print(snippet["text"])
                context += snippet["text"]  
            return(context)

In [22]:
def getSnippets(id):
    for quesion in data['questions']:
        if(quesion['type'] == "factoid" and quesion["id"] == id):  
            return(quesion["snippets"])

In [16]:

paragraphs = []
for quesion in data['questions']:
    if(quesion['type'] == "factoid"):        
        id = quesion["id"]
        snippets = getSnippets(id)
        for snippet in snippets: 
            answers = []
            answerContent = {}
            #text = quesion["exact_answer"][0]
            answer_start = 0
            #answerContent["text"] = text
            answerContent["answer_start"] = answer_start
            answers.append(answerContent)

            qas = []
            qasContent = {}
            id = quesion["id"]
            question = quesion["body"] 
            qasContent['question'] = question
            qasContent['id'] = id
            qasContent['answers'] = answers
            qas.append(qasContent)

            paragraphContent = {}
            paragraphContent["context"] = snippet
            paragraphContent["qas"] = qas
            paragraphs.append(paragraphContent)


### convert the answers in BIOASQ format

In [95]:
with open('/Users/aditya/Documents/Spring_2019/BioAsq/7B/Batch5/BioASQ-task7bPhaseB-testset5.json') as f:
    data_upload = json.load(f)


In [96]:
# #dict1["exact_answer"] = ["Dummy"]
# arr = []
# for question in data_upload['questions']:
#     if(question["type"] != "factoid" and question["type"] != "list"):
#         dict1 = {}
#         if(question["type"] == "yesno"):
#             dict1["exact_answer"] = "yes"
#         else:
#             dict1["exact_answer"] = ["Dummy"]
#         dict1["ideal_answer"] = ["Dummy"]
#         dict1["type"] = question["type"] 
#         dict1["id"] = question["id"]
#         arr.append(json.dumps(dict1))
#dict1["exact_answer"] = ["Dummy"]
arr = []
for question in data_upload['questions']:
    if(question["type"] != "factoid" and question["type"] != "list"):
        dict1 = {}
        if(question["type"] == "yesno"):
            dict1["exact_answer"] = "yes"
        else:
            dict1["exact_answer"] = "Dummy"
        dict1["ideal_answer"] = "Dummy"
        dict1["type"] = question["type"] 
        dict1["id"] = question["id"]
        arr.append(json.dumps(dict1))



In [97]:
count = 0
for i in arr:
        print(i+",")
        count += 1        

{"exact_answer": "Dummy", "ideal_answer": "Dummy", "type": "summary", "id": "5ca60a77ecadf2e73f00004d"},
{"exact_answer": "Dummy", "ideal_answer": "Dummy", "type": "summary", "id": "5c88b8a275a4a5d21900000b"},
{"exact_answer": "yes", "ideal_answer": "Dummy", "type": "yesno", "id": "5c97a08becadf2e73f000029"},
{"exact_answer": "yes", "ideal_answer": "Dummy", "type": "yesno", "id": "5c8ab614d558e5f23200000d"},
{"exact_answer": "yes", "ideal_answer": "Dummy", "type": "yesno", "id": "5ca61176ecadf2e73f00004e"},
{"exact_answer": "yes", "ideal_answer": "Dummy", "type": "yesno", "id": "5c8908a475a4a5d21900000c"},
{"exact_answer": "yes", "ideal_answer": "Dummy", "type": "yesno", "id": "5c98ac7fecadf2e73f00002b"},
{"exact_answer": "yes", "ideal_answer": "Dummy", "type": "yesno", "id": "5c890c3375a4a5d21900000e"},
{"exact_answer": "yes", "ideal_answer": "Dummy", "type": "yesno", "id": "5ca0c81eecadf2e73f000047"},
{"exact_answer": "Dummy", "ideal_answer": "Dummy", "type": "summary", "id": "5ca9f5

In [98]:
count

53

In [99]:
100 - 53

47

In [100]:
35 + 12

47